Connected to Python 3.10.11

In [3]:
import sys
import os
import itertools
import matplotlib.pyplot as plt
import pandas as pd

# Set random seed for reproducibility
import random
random.seed(42)

# Set paths
notebook_dir = os.getcwd()
src_path = os.path.abspath(os.path.join(notebook_dir, '..', 'src'))
if src_path not in sys.path:
    sys.path.append(src_path)

from utils import add_src_to_path
add_src_to_path()

from data_split import split_time_series_by_river
from model import train_and_apply_with_tuning

from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_percentage_error

# Experiment parameters
river_systems = ["Fraser River", "Bristol Bay", "Columbia River", "Bristol Bay - Ugashik"]
top_k_options = [6, 10, 20, 0]  # 0 means use all features
add_feat_options = [True, False]
arima_options = [True, False]

project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
data_path = os.path.join(project_root, 'data', 'Combined_FeatureSet_For_Model.csv')
output_path = os.path.abspath(os.path.join(notebook_dir, '..', 'results', 'Model_Performances.csv'))


model_list = ["RF", "GBRT", "XGB", "LR", "PR"]
all_experiments = []

for river_system, top_k_features, add_additional_features, use_arima_on_top in itertools.product(
    river_systems, top_k_options, add_feat_options, arima_options
):
    print(f"\n Running: {river_system} | TopK={top_k_features} | ExtraFeat={add_additional_features} | ARIMA={use_arima_on_top}")

    combined_df = pd.read_csv(data_path)

    # Drop features that have missing values in 2024
    columns_to_drop = [
        'mean_temp_JunAug', 'mean_temp_MaySep', 'max_temp_JunAug',
        'mean_discharge_MarMay', 'max_discharge_AprSep'
    ]
    combined_df = combined_df.drop(columns=[col for col in columns_to_drop if col in combined_df.columns])

    # Setup special rules for river Ugashik
    if river_system == "Bristol Bay - Ugashik":
        river_system = "Bristol Bay"
        apply_strict_dropna = False # Delets the spawner data for Bristol Bay
    else:
        apply_strict_dropna = True

    combined_df = combined_df[combined_df["System"] == river_system]

    # Feature engineering  
    if add_additional_features:
        features_to_lag = [
            'Total_Returns', 'AgeClass_0.1', 'AgeClass_0.2', 'AgeClass_0.3', 'AgeClass_0.4', 'AgeClass_0.5',
            'AgeClass_1.1', 'AgeClass_1.2', 'AgeClass_1.3', 'AgeClass_1.4', 'AgeClass_1.5',
            'AgeClass_2.1', 'AgeClass_2.2', 'AgeClass_2.3', 'AgeClass_2.4',
            'AgeClass_3.1', 'AgeClass_3.2', 'AgeClass_3.3', 'AgeClass_3.4',
            'Total_Returns_NextYear', 'Pacea_ALPI_Anomaly', 'npi_mean_NovMar', 'oni_mean_DecFeb',
            'npgo_mean_DecFeb', 'ao_mean_DecMar', 'pdo_mean_DecMar', 'pdo_mean_MaySep'
        ]
        for feat in features_to_lag:
            for lag in [1, 2, 3, 4, 5]:
                combined_df[f'{feat}_Yminus{lag}'] = combined_df.groupby(['System', 'River'])[feat].shift(lag)

    if (apply_strict_dropna and river_system == "Bristal Bay") or river_system == "Columbia River":
        # If Columbia River or Bristol Bay, only keep lines with spawner data
        combined_df = combined_df.dropna(subset=['total_spawners_y_minus_2_to_4'])
    if add_additional_features:
        combined_df = combined_df.dropna(subset=['AgeClass_0.2_Yminus5'])

    # If system is Bristol Bay, consider only years after 1995
    if river_system == "Bristol Bay":
        combined_df = combined_df[combined_df["Year"] >= 1995]

    # Drop features with any NaN values
    combined_df = combined_df.dropna(axis=1, how='any').dropna()

    train_df, test_df = split_time_series_by_river(
        combined_df,
        time_column="Year",
        group_columns=["System", "River"],
        test_fraction=0.2,
        gap_years=0
    )

    train_df["River_Name"] = train_df["River"]
    test_df["River_Name"] = test_df["River"]

    train_df_encoded = pd.get_dummies(train_df, columns=["River"], prefix="River")
    test_df_encoded = pd.get_dummies(test_df, columns=["River"], prefix="River")

    for model_name in model_list:
        try:
            results = train_and_apply_with_tuning(
                model=model_name,
                train_df=train_df_encoded,
                test_df=test_df_encoded,
                topk_feat=top_k_features
            )

            # Apply ARIMA correction if selected
            if use_arima_on_top:
                residuals = results["Timeline_train"]["Actual"] - results["Timeline_train"]["Predicted"]
                residuals_series = pd.Series(residuals.values, index=results["Timeline_train"]["Year"])

                arima_model = ARIMA(residuals_series, order=(1, 0, 0))
                arima_fit = arima_model.fit()

                residual_forecast = arima_fit.forecast(steps=len(results["Timeline_test"]["Predicted"]))
                hybrid_pred = results["Timeline_test"]["Predicted"] + residual_forecast.values

                results["Timeline_test"]["Predicted"] = hybrid_pred

                # Optionally update metrics
                r2 = r2_score(results["Timeline_test"]["Actual"], hybrid_pred)
                mse = mean_squared_error(results["Timeline_test"]["Actual"], hybrid_pred)
                mape = mean_absolute_percentage_error(results["Timeline_test"]["Actual"], hybrid_pred) * 100

                # Also update metrics per river (recalculate)
                grouped = results["Timeline_test"].groupby("River_Name")
                metrics_by_river_test = grouped.apply(
                    lambda g: pd.Series({
                        "R2": r2_score(g["Actual"], g["Predicted"]),
                        "MSE": mean_squared_error(g["Actual"], g["Predicted"]),
                        "MAPE": mean_absolute_percentage_error(g["Actual"], g["Predicted"]) * 100
                    })
                ).reset_index()

                results["Metrics_by_River_Test"] = metrics_by_river_test

            # Prepare final output
            test_df_ = results['Metrics_by_River_Test'].copy()
            train_df_ = results['Metrics_by_River_Train'].copy()

            test_df_ = test_df_.rename(columns={"R2": "R2_Test", "MSE": "MSE_Test", "MAPE": "MAPE_Test"})
            train_df_ = train_df_.rename(columns={"R2": "R2_Train", "MSE": "MSE_Train", "MAPE": "MAPE_Train"})

            merged_df = pd.merge(test_df_, train_df_, on="River_Name", how="outer")
            merged_df.insert(0, "Model", model_name)

            river_system_lookup = pd.concat([train_df_encoded, test_df_encoded])["River_Name"].drop_duplicates()
            river_system_lookup = pd.merge(
                river_system_lookup.to_frame(),
                combined_df[["River", "System"]].drop_duplicates(),
                left_on="River_Name",
                right_on="River",
                how="left"
            ).drop(columns="River")

            merged_df = pd.merge(merged_df, river_system_lookup, on="River_Name", how="left")

            merged_df["Selected_System"] = river_system
            merged_df["TopK_Features"] = top_k_features
            merged_df["Additional_Features_Used"] = add_additional_features
            merged_df["ARIMA_Enabled"] = use_arima_on_top

            feature_names = results.get("Selected_Feature_Names", [])
            merged_df["Num_Features_Used"] = len(feature_names)
            merged_df["Feature_Names"] = ", ".join(feature_names)

            all_experiments.append(merged_df)

        except Exception as e:
            print(f"Error with model {model_name}: {e}")

# Save final results
final_df = pd.concat(all_experiments, ignore_index=True)
final_df.to_csv(output_path, index=False)

print(f"\n All experiment results saved to:\n{output_path}")



 Running: Fraser River | TopK=6 | ExtraFeat=True | ARIMA=True
Selected features:
Index(['AgeClass_0.2', 'total_spawners_y_minus_3', 'Total_Returns_Yminus3',
       'AgeClass_1.1_Yminus4', 'AgeClass_1.2_Yminus3',
       'Total_Returns_NextYear_Yminus4'],
      dtype='object')


C:\Users\julia\AppData\Local\Temp\ipykernel_30120\3055512051.py:75: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  combined_df[f'{feat}_Yminus{lag}'] = combined_df.groupby(['System', 'River'])[feat].shift(lag)
C:\Users\julia\AppData\Local\Temp\ipykernel_30120\3055512051.py:75: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  combined_df[f'{feat}_Yminus{lag}'] = combined_df.groupby(['System', 'River'])[feat].shift(lag)
C:\Users\julia\AppData\Local\Temp\ipykernel_30120\3055512051.py:75: PerformanceWarning: DataFrame is highly fragment

Best Parameters: {'max_depth': 10, 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 100}
RF R2: 0.40
RF MSE: 248294181626.38
RF MAPE: 431.23
Selected features:
Index(['AgeClass_0.2', 'total_spawners_y_minus_3', 'Total_Returns_Yminus3',
       'AgeClass_1.1_Yminus4', 'AgeClass_1.2_Yminus3',
       'Total_Returns_NextYear_Yminus4'],
      dtype='object')


C:\Users\julia\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: An unsupported index was provided. As a result, forecasts cannot be generated. To use the model for forecasting, use one of the supported classes of index.
  self._init_dates(dates, freq)
C:\Users\julia\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: An unsupported index was provided. As a result, forecasts cannot be generated. To use the model for forecasting, use one of the supported classes of index.
  self._init_dates(dates, freq)
C:\Users\julia\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: An unsupported index was provided. As a result, 

Best Parameters: {'learning_rate': 0.05, 'max_depth': 5, 'max_iter': 100, 'min_samples_leaf': 1}
GBRT R2: 0.01
GBRT MSE: 409256660172.12
GBRT MAPE: 421.91
Selected features:
Index(['AgeClass_0.2', 'total_spawners_y_minus_3', 'Total_Returns_Yminus3',
       'AgeClass_1.1_Yminus4', 'AgeClass_1.2_Yminus3',
       'Total_Returns_NextYear_Yminus4'],
      dtype='object')


...

Best Parameters: {'fit_intercept': True, 'positive': False}
LR R2: 0.48
LR MSE: 10788925069424.01
LR MAPE: 43.97


C:\Users\julia\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: An unsupported index was provided. As a result, forecasts cannot be generated. To use the model for forecasting, use one of the supported classes of index.
  self._init_dates(dates, freq)
C:\Users\julia\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: An unsupported index was provided. As a result, forecasts cannot be generated. To use the model for forecasting, use one of the supported classes of index.
  self._init_dates(dates, freq)
C:\Users\julia\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: An unsupported index was provided. As a result, 

Selected features:
Index(['Total_Returns', 'AgeClass_0.1', 'AgeClass_0.2', 'AgeClass_0.3',
       'AgeClass_0.4', 'AgeClass_0.5', 'AgeClass_1.1', 'AgeClass_1.2',
       'AgeClass_1.3', 'AgeClass_1.4', 'AgeClass_1.5', 'AgeClass_2.1',
       'AgeClass_2.2', 'AgeClass_2.3', 'AgeClass_2.4', 'AgeClass_3.1',
       'AgeClass_3.2', 'AgeClass_3.3', 'AgeClass_3.4', 'Pacea_ALPI_Anomaly',
       'npi_mean_NovMar', 'oni_mean_DecFeb', 'mei_mean_AprSep',
       'npgo_mean_DecFeb', 'ao_mean_DecMar', 'pdo_mean_DecMar',
       'pdo_mean_MaySep', 'sst_aprjul', 'sst_anom', 'sss_mayaug',
       'River_Alagnak', 'River_Egegik', 'River_Igushik', 'River_Kvichak',
       'River_Naknek', 'River_Nushagak', 'River_Ugashik', 'River_Wood'],
      dtype='object')
No parameter tuning for this model.
PR R2: -32379.33
PR MSE: 674950157888640896.00
PR MAPE: 2322.82


C:\Users\julia\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: An unsupported index was provided. As a result, forecasts cannot be generated. To use the model for forecasting, use one of the supported classes of index.
  self._init_dates(dates, freq)
C:\Users\julia\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: An unsupported index was provided. As a result, forecasts cannot be generated. To use the model for forecasting, use one of the supported classes of index.
  self._init_dates(dates, freq)
C:\Users\julia\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: An unsupported index was provided. As a result, 


 Running: Bristol Bay - Ugashik | TopK=0 | ExtraFeat=False | ARIMA=False
Selected features:
Index(['Total_Returns', 'AgeClass_0.1', 'AgeClass_0.2', 'AgeClass_0.3',
       'AgeClass_0.4', 'AgeClass_0.5', 'AgeClass_1.1', 'AgeClass_1.2',
       'AgeClass_1.3', 'AgeClass_1.4', 'AgeClass_1.5', 'AgeClass_2.1',
       'AgeClass_2.2', 'AgeClass_2.3', 'AgeClass_2.4', 'AgeClass_3.1',
       'AgeClass_3.2', 'AgeClass_3.3', 'AgeClass_3.4', 'Pacea_ALPI_Anomaly',
       'npi_mean_NovMar', 'oni_mean_DecFeb', 'mei_mean_AprSep',
       'npgo_mean_DecFeb', 'ao_mean_DecMar', 'pdo_mean_DecMar',
       'pdo_mean_MaySep', 'sst_aprjul', 'sst_anom', 'sss_mayaug',
       'River_Alagnak', 'River_Egegik', 'River_Igushik', 'River_Kvichak',
       'River_Naknek', 'River_Nushagak', 'River_Ugashik', 'River_Wood'],
      dtype='object')
Best Parameters: {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 100}
RF R2: 0.52
RF MSE: 10071513942524.17
RF MAPE: 34.63
Selected features:
Index